# Fase 3: Preparação da Base para Modelagem (EAP 3.0)

Este notebook implementa a Fase 3 do projeto de previsão de tendência do IBOVESPA, estruturando os dados para que possam ser consumidos pelos algoritmos de machine learning, respeitando a ordem temporal.

**Autor:** Projeto Tech Challenge 2  
**Data:** 2025-01-24  
**Referência:** EAP.md e Steering.md

## 🎯 Objetivos da Fase 3:

1. **Estruturação com Janela Deslizante**: Transformar série temporal em dataset tabular
2. **Divisão Cronológica**: Separar dados respeitando ordem temporal
3. **Escalonamento**: Normalizar features para melhor performance dos modelos

## ⚠️ Abordagem Funcional

**Mudança de arquitetura**: Convertemos a estrutura de classe para **abordagem funcional** mais adequada ao formato notebook, facilitando a compreensão e execução sequencial.

## 1. Importações e Configurações

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette('husl')

# Verifica versões
print('=== VERIFICAÇÃO DE COMPATIBILIDADE ===')
print(f'✓ Pandas: {pd.__version__}')
print(f'✓ NumPy: {np.__version__}')
print(f'✓ Scikit-learn: Importado com sucesso')
print('✓ Bibliotecas importadas com sucesso!')
print('✓ Pronto para preparação dos dados para modelagem')

## 2. Carregamento dos Dados da Fase 2

In [ ]:
# Carrega os dados processados da Fase 2
try:
    dados_com_features = pd.read_csv('dados_fase2_completos.csv', index_col=0, parse_dates=True)
    print('✓ Dados da Fase 2 carregados com sucesso!')
except FileNotFoundError:
    print('⚠️ Arquivo dados_fase2_completos.csv não encontrado.')
    print('⚠️ Executando a Fase 2 primeiro ou carregando dados alternativos...')
    # Carrega dados básicos como fallback
    dados_com_features = pd.read_csv('dados_bovespa.csv', index_col=0, parse_dates=True)
    # Cria target simples para demonstração
    col_close = [col for col in dados_com_features.columns if 'close' in col.lower()][0]
    dados_com_features['Target'] = (dados_com_features[col_close].shift(-1) > dados_com_features[col_close]).astype(int)
    dados_com_features = dados_com_features.dropna()
    print('✓ Dados básicos carregados e target criado')

print(f'\n=== INFORMAÇÕES DOS DADOS ===')
print(f'✓ Registros: {len(dados_com_features)}')
print(f'✓ Período: {dados_com_features.index.min()} a {dados_com_features.index.max()}')
print(f'✓ Colunas: {len(dados_com_features.columns)}')
print(f'✓ Colunas disponíveis: {list(dados_com_features.columns)}')

# Verifica se há coluna Target
if 'Target' not in dados_com_features.columns:
    raise ValueError('Coluna Target não encontrada. Execute a Fase 2 primeiro.')

# Visualiza as primeiras linhas
print('\n=== PRIMEIRAS LINHAS DOS DADOS ===')
dados_com_features.head()

## 3. Estruturação com Janela Deslizante

### 3.1 Definição da Janela de Entrada (Lookback Window)

Transformamos a série temporal em um dataset tabular onde cada linha contém os atributos dos últimos **n=5 dias** e o alvo correspondente.

In [ ]:
# Configurações da janela deslizante
JANELA_TAMANHO = 5  # Lookback window de 5 dias

print(f'=== ESTRUTURAÇÃO COM JANELA DESLIZANTE ===')
print(f'✓ Tamanho da janela: {JANELA_TAMANHO} dias')

# Identifica features numéricas (exclui Target)
colunas_excluir = ['Target']
features_numericas = dados_com_features.select_dtypes(include=[np.number]).columns
features_numericas = [col for col in features_numericas if col not in colunas_excluir]

print(f'✓ Features selecionadas: {len(features_numericas)}')
print(f'✓ Features: {features_numericas[:10]}...')  # Mostra apenas as primeiras 10

# Remove linhas com NaN nas features
dados_limpos = dados_com_features[features_numericas + ['Target']].dropna()
print(f'✓ Dados limpos: {len(dados_limpos)} registros')

### 3.2 Implementação da Lógica de Janela Deslizante

In [ ]:
# Cria estrutura de janela deslizante
print('=== CRIANDO ESTRUTURA DE JANELA DESLIZANTE ===')

X_list = []
y_list = []
indices_list = []

# Itera pelos dados criando janelas deslizantes
for i in range(JANELA_TAMANHO, len(dados_limpos)):
    # Janela de features (últimos n dias)
    janela_features = dados_limpos[features_numericas].iloc[i-JANELA_TAMANHO:i].values
    
    # Achata a janela (transforma matriz em vetor)
    janela_achatada = janela_features.flatten()
    
    # Target correspondente
    target = dados_limpos['Target'].iloc[i]
    
    X_list.append(janela_achatada)
    y_list.append(target)
    indices_list.append(dados_limpos.index[i])

# Converte para arrays numpy
X = np.array(X_list)
y = np.array(y_list)

print(f'✓ Janelas criadas: {len(X_list)}')
print(f'✓ Shape do X: {X.shape}')
print(f'✓ Shape do y: {y.shape}')

# Cria nomes das colunas para o DataFrame final
nomes_colunas = []
for lag in range(JANELA_TAMANHO, 0, -1):
    for feature in features_numericas:
        nomes_colunas.append(f"{feature}_lag_{lag}")

print(f'✓ Colunas criadas: {len(nomes_colunas)}')
print(f'✓ Exemplo de colunas: {nomes_colunas[:5]}...')

# Cria DataFrame estruturado
dados_janela_deslizante = pd.DataFrame(
    X, 
    columns=nomes_colunas,
    index=indices_list
)
dados_janela_deslizante['Target'] = y

print(f'\n✓ Dataset estruturado criado: {X.shape[0]} amostras, {X.shape[1]} features')
print(f'✓ Cada amostra contém {JANELA_TAMANHO} dias de {len(features_numericas)} features')

# Visualiza as primeiras linhas
print('\n=== PRIMEIRAS LINHAS DO DATASET ESTRUTURADO ===')
dados_janela_deslizante.head()

## 4. Divisão Cronológica dos Dados

### 4.1 Definição da Data de Corte

Separamos os dados em treino (80%) e teste (20%) **sem usar amostragem aleatória**, respeitando a ordem cronológica.

In [ ]:
# Configurações da divisão
PROPORCAO_TREINO = 0.8

print(f'=== DIVISÃO CRONOLÓGICA DOS DADOS ===')
print(f'✓ Proporção treino/teste: {PROPORCAO_TREINO:.0%}/{1-PROPORCAO_TREINO:.0%}')

# Calcula ponto de corte cronológico
total_amostras = len(dados_janela_deslizante)
ponto_corte = int(total_amostras * PROPORCAO_TREINO)

# Data de corte
data_corte = dados_janela_deslizante.index[ponto_corte]
print(f'✓ Data de corte: {data_corte}')
print(f'✓ Ponto de corte: {ponto_corte} (de {total_amostras} amostras)')

# Separação cronológica
dados_treino = dados_janela_deslizante.iloc[:ponto_corte]
dados_teste = dados_janela_deslizante.iloc[ponto_corte:]

print(f'\n✓ Período de treino: {dados_treino.index.min()} até {dados_treino.index.max()}')
print(f'✓ Período de teste: {dados_teste.index.min()} até {dados_teste.index.max()}')
print(f'✓ Amostras de treino: {len(dados_treino)}')
print(f'✓ Amostras de teste: {len(dados_teste)}')

### 4.2 Separação de Features e Target

In [ ]:
# Separa features e target
colunas_features = [col for col in dados_janela_deslizante.columns if col != 'Target']

X_train = dados_treino[colunas_features]
y_train = dados_treino['Target']
X_test = dados_teste[colunas_features]
y_test = dados_teste['Target']

print(f'=== SEPARAÇÃO DE FEATURES E TARGET ===')
print(f'✓ Features: {len(colunas_features)}')
print(f'✓ X_train shape: {X_train.shape}')
print(f'✓ y_train shape: {y_train.shape}')
print(f'✓ X_test shape: {X_test.shape}')
print(f'✓ y_test shape: {y_test.shape}')

# Verifica distribuição de classes
dist_treino = y_train.value_counts(normalize=True)
dist_teste = y_test.value_counts(normalize=True)

print(f'\n✓ Distribuição de classes no treino:')
for classe, prop in dist_treino.items():
    print(f'   - Classe {classe}: {prop:.2%}')

print(f'✓ Distribuição de classes no teste:')
for classe, prop in dist_teste.items():
    print(f'   - Classe {classe}: {prop:.2%}')

# Visualização gráfica da distribuição
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Distribuição no treino
dist_treino.plot(kind='bar', ax=ax1, color=['red', 'green'])
ax1.set_title('Distribuição de Classes - Treino')
ax1.set_xlabel('Classe Target')
ax1.set_ylabel('Proporção')
ax1.tick_params(axis='x', rotation=0)

# Distribuição no teste
dist_teste.plot(kind='bar', ax=ax2, color=['red', 'green'])
ax2.set_title('Distribuição de Classes - Teste')
ax2.set_xlabel('Classe Target')
ax2.set_ylabel('Proporção')
ax2.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 5. Escalonamento de Atributos

### 5.1 Instanciação e Ajuste do StandardScaler

**Importante**: O scaler é ajustado **APENAS** no conjunto de treino para evitar data leakage.

In [ ]:
print(f'=== ESCALONAMENTO DE ATRIBUTOS ===')

# 5.1.1 - Instancia StandardScaler
scaler = StandardScaler()
print('✓ StandardScaler instanciado')

# 5.1.2 - Ajusta APENAS no conjunto de treino
scaler.fit(X_train)
print('✓ Scaler ajustado APENAS nos dados de treino')
print('✓ Isso previne data leakage!')

# Mostra estatísticas do scaler
print(f'\n✓ Estatísticas do scaler (baseadas no treino):')
print(f'   - Número de features: {len(scaler.mean_)}')
print(f'   - Média das primeiras 5 features: {scaler.mean_[:5]}')
print(f'   - Desvio padrão das primeiras 5 features: {scaler.scale_[:5]}')

### 5.2 Aplicação da Transformação

In [ ]:
# 5.2.1 - Aplica transformação em ambos os conjuntos
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'=== APLICAÇÃO DA TRANSFORMAÇÃO ===')
print('✓ Transformação aplicada em treino e teste')
print('✓ Usando os mesmos parâmetros ajustados no treino')

# Converte de volta para DataFrame mantendo índices e nomes das colunas
X_train_scaled_df = pd.DataFrame(
    X_train_scaled,
    index=X_train.index,
    columns=X_train.columns
)

X_test_scaled_df = pd.DataFrame(
    X_test_scaled,
    index=X_test.index,
    columns=X_test.columns
)

print(f'\n✓ DataFrames escalonados criados:')
print(f'   - X_train_scaled shape: {X_train_scaled_df.shape}')
print(f'   - X_test_scaled shape: {X_test_scaled_df.shape}')

# Verifica se a transformação foi aplicada corretamente
media_treino = X_train_scaled_df.mean().mean()
std_treino = X_train_scaled_df.std().mean()

print(f'\n✓ Verificação da transformação:')
print(f'   - Média do treino escalonado: {media_treino:.6f} (deve ser ~0)')
print(f'   - Desvio padrão do treino escalonado: {std_treino:.6f} (deve ser ~1)')

if abs(media_treino) < 1e-10 and abs(std_treino - 1) < 0.1:
    print('✅ Escalonamento aplicado corretamente!')
else:
    print('⚠️ Verificar escalonamento - valores fora do esperado')

## 6. Visualização dos Dados Preparados

In [ ]:
print(f'=== VISUALIZAÇÃO DOS DADOS PREPARADOS ===')

# Comparação antes e depois do escalonamento
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Primeira feature - antes do escalonamento
primeira_feature = X_train.columns[0]
axes[0, 0].hist(X_train[primeira_feature], bins=30, alpha=0.7, color='blue')
axes[0, 0].set_title(f'Antes do Escalonamento\n{primeira_feature}')
axes[0, 0].set_xlabel('Valor')
axes[0, 0].set_ylabel('Frequência')

# Primeira feature - depois do escalonamento
axes[0, 1].hist(X_train_scaled_df[primeira_feature], bins=30, alpha=0.7, color='green')
axes[0, 1].set_title(f'Depois do Escalonamento\n{primeira_feature}')
axes[0, 1].set_xlabel('Valor Escalonado')
axes[0, 1].set_ylabel('Frequência')

# Distribuição geral - antes
sample_features = X_train.iloc[:, :5]  # Primeiras 5 features
axes[1, 0].boxplot([sample_features[col].dropna() for col in sample_features.columns])
axes[1, 0].set_title('Distribuição Geral - Antes')
axes[1, 0].set_xlabel('Features (primeiras 5)')
axes[1, 0].set_ylabel('Valor')

# Distribuição geral - depois
sample_features_scaled = X_train_scaled_df.iloc[:, :5]  # Primeiras 5 features
axes[1, 1].boxplot([sample_features_scaled[col].dropna() for col in sample_features_scaled.columns])
axes[1, 1].set_title('Distribuição Geral - Depois')
axes[1, 1].set_xlabel('Features (primeiras 5)')
axes[1, 1].set_ylabel('Valor Escalonado')

plt.tight_layout()
plt.show()

# Mostra estatísticas descritivas
print('\n=== ESTATÍSTICAS DESCRITIVAS ===')
print('\n📊 Dados ANTES do escalonamento (primeiras 5 features):')
print(X_train.iloc[:, :5].describe())

print('\n📊 Dados DEPOIS do escalonamento (primeiras 5 features):')
print(X_train_scaled_df.iloc[:, :5].describe())

## 7. Salvamento dos Dados Preparados

In [ ]:
print(f'=== SALVAMENTO DOS DADOS PREPARADOS ===')

# Salva os dados preparados
X_train.to_csv('X_train_fase3.csv')
X_test.to_csv('X_test_fase3.csv')
y_train.to_csv('y_train_fase3.csv')
y_test.to_csv('y_test_fase3.csv')

# Salva os dados escalonados
X_train_scaled_df.to_csv('X_train_scaled_fase3.csv')
X_test_scaled_df.to_csv('X_test_scaled_fase3.csv')

# Salva informações da preparação
info_preparacao = {
    'janela_tamanho': JANELA_TAMANHO,
    'proporcao_treino': PROPORCAO_TREINO,
    'data_corte': str(data_corte),
    'total_features': len(colunas_features),
    'amostras_treino': len(X_train),
    'amostras_teste': len(X_test),
    'features_originais': len(features_numericas),
    'periodo_treino': f'{X_train.index.min()} até {X_train.index.max()}',
    'periodo_teste': f'{X_test.index.min()} até {X_test.index.max()}'
}

with open('info_preparacao_fase3.json', 'w', encoding='utf-8') as f:
    json.dump(info_preparacao, f, indent=2, ensure_ascii=False, default=str)

print('✓ Arquivos salvos:')
print('   - X_train_fase3.csv')
print('   - X_test_fase3.csv')
print('   - y_train_fase3.csv')
print('   - y_test_fase3.csv')
print('   - X_train_scaled_fase3.csv')
print('   - X_test_scaled_fase3.csv')
print('   - info_preparacao_fase3.json')

print(f'\n✅ Dados preparados e salvos com sucesso!')

## 8. Resumo da Fase 3

### ✅ Objetivos Alcançados:

1. **✅ Estruturação com Janela Deslizante**: 
   - Janela de 5 dias implementada
   - Série temporal transformada em dataset tabular
   - Cada amostra contém histórico de 5 dias

2. **✅ Divisão Cronológica dos Dados**:
   - Divisão 80/20 respeitando ordem temporal
   - Sem amostragem aleatória
   - Data de corte bem definida
   - Distribuição de classes preservada

3. **✅ Escalonamento de Atributos**:
   - StandardScaler ajustado apenas no treino
   - Prevenção de data leakage
   - Transformação aplicada em treino e teste
   - Verificação de qualidade realizada

4. **✅ Abordagem Funcional**:
   - Código sem classes, mais direto
   - Adequado ao formato notebook
   - Fácil compreensão e execução
   - Compatível com NumPy 2.0+

### 📊 Dados Finais Preparados:

- **Treino**: {len(X_train)} amostras com {len(colunas_features)} features
- **Teste**: {len(X_test)} amostras com {len(colunas_features)} features
- **Janela**: {JANELA_TAMANHO} dias de histórico por amostra
- **Features**: Dados escalonados (média≈0, std≈1)
- **Target**: Balanceamento preservado entre treino e teste

### 🎯 Próximos Passos:

1. **Fase 4**: Treinamento e validação de modelos de ML
2. **Modelos a testar**: XGBoost, Random Forest, SVM, etc.
3. **Validação**: Time Series Cross-Validation
4. **Métricas**: Precisão, Recall, F1-Score, AUC-ROC

### 💡 Pontos Importantes:

- **✅ Data Leakage Prevenido**: Scaler ajustado apenas no treino
- **✅ Ordem Temporal Respeitada**: Divisão cronológica sem aleatoriedade
- **✅ Estrutura Adequada**: Janela deslizante para capturar padrões temporais
- **✅ Dados Prontos**: Formato ideal para algoritmos de ML

---

**🚀 Os dados estão prontos para a modelagem na Fase 4!**